התקנת ספרייה של גוגל לשימוש במנוע AI

In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 7.7 MB/s eta 0:00:00


ספריות רלוונטיות שהשתמשנו בהם

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('.',' *')
  return Markdown(textwrap.indent(text,'>', predicate= lambda _:True))

מפתח סודי לשימוש בבינה המלכותית בקוד

In [ ]:
GOOGLE_API_KEY = userdata.get('gemini_key')

genai.configure(api_key = GOOGLE_API_KEY)

רשימת המודלים הזמינים לשימוש

In [ ]:
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)


models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


בחירת המודל

In [ ]:
model = genai.GenerativeModel('gemini-pro')

הפקודה שמאפשרת לעבוד עם קבצי ווארד בפייתון

In [ ]:
!pip install python-docx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 1.9 MB/s eta 0:00:00


הסקריפט ששולח מאמרים ושאלות לבינה המלכותית ומחזיר תשובות בקובץ אקסל

In [ ]:
import openpyxl
from openpyxl import Workbook
import docx
import re
import time

# קריאת קובץ Word והוצאת הטקסט
def read_word_file(file_path):
    doc = docx.Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

# השאלות שיישלחו למודל גימיני
questions = """
1. How much does the sea level rise every year?
2. How much has the global sea level risen in total over the past 50 years in centimeters?
3. What is the projected sea level rise by the year 2100 according to different climate models in centimeters or meters?
4. How much have specific coastal cities (e.g., New York, Miami, Tokyo) experienced in terms of sea level rise in the past 50 years in centimeters?
5. What percentage of global ice melt (from glaciers and ice sheets) contributes to sea level rise annually?
6. How many people are estimated to be at risk of displacement due to sea level rise by 2050 and 2100?
7. What is the average rate of coastal erosion per year in regions most affected by sea level rise in meters?
8. What is the correlation between global temperature increase and sea level rise over the past century?
9. What is the estimated volume of water added to the oceans annually from melting Greenland and Antarctic ice sheets?
10.How much has the sea level rise accelerated (in percentage) over the last decade compared to the historical average?
"""

# פונקציה לשליחת בקשה לגימיני עם ניסיון מחדש
def generate_response_with_retry(abstract, questions, retries=3):
    for attempt in range(retries):
        try:
            # שליחת בקשה למודל
            response = model.generate_content(
                f"Based on this abstract: '{abstract}', please answer these questions: {questions}. If the abstract does not provide information for a question, respond with 'NULL'."
            )
            return response
        except Exception as e:
            print(f"Error on attempt {attempt + 1}: {e}")
            if "429" in str(e):  # קוד שגיאה למניעת עומס
                time.sleep(60)  # המתנה של 60 שניות
            else:
                time.sleep(15)  # המתנה של 15 שניות
    return None

file_path = "/content/SEA LEVEL RISE ABS.docx"

# קריאת תוכן מקובץ ה-Word
try:
    content = read_word_file(file_path)
except Exception as e:
    print(f"Failed to read the Word file: {e}")
    content = ""

# חלוקת הטקסט למאמרים לפי הכותרת של כל מאמר
articles = re.split(r'(?=\bTitle:\s)', content)

# יצירת רשימה לאחסון המאמרים
abstracts_array = []

for article in articles:
    record_info = {}

    # מציאת הכותרת
    title_match = re.search(r'Title:\s*(.*)', article)
    if title_match:
        record_info['Title'] = title_match.group(1).strip()

    # מציאת השנה
    year_match = re.search(r'Published:\s*(\d{4})', article)
    if year_match:
        record_info['Year'] = year_match.group(1).strip()

    # מציאת התקציר
    abstract_match = re.search(r'Abstract:\s*(.*)', article, re.DOTALL)
    if abstract_match:
        record_info['Abstract'] = abstract_match.group(1).strip()

    # אם מצאנו גם כותרת וגם תקציר, נוסיף את המאמר לרשימה
    if 'Title' in record_info and 'Abstract' in record_info:
        abstracts_array.append(record_info)

# הצגת מספר המאמרים שמצאנו
print(f"Total articles found: {len(abstracts_array)}")

# יצירת קובץ אקסל חדש
wb = Workbook()
ws = wb.active
ws.title = "Article Responses"

# כתיבת כותרות העמודות
headers = ["Title", "Year", "Abstract"] + [f"Q{i}" for i in range(1, 11)]
ws.append(headers)

# ריצה על כל המאמרים ושמירת התשובות
for article in abstracts_array:
    abstract = article.get("Abstract", "")
    response = generate_response_with_retry(abstract, questions)

    if response:
        # הנחת שהתשובה מגיעה באובייקט של המודל ויש לגשת לתוכן שלו
        try:
            # בדיקה אם האובייקט מכיל טקסט בתור חלק יחיד או בתור חלקים
            if hasattr(response, 'text'):
                answers = response.text.strip().split('\n')
            elif hasattr(response, 'parts') and response.parts:
                # שילוב כל החלקים יחד לפסקה אחת, ואז הפרדה לפי שורות
                combined_text = '\n'.join(part.text for part in response.parts if hasattr(part, 'text'))
                answers = combined_text.strip().split('\n')
            else:
                answers = ["NULL"] * 10
        except Exception as e:
            print(f"Failed to parse response: {e}")
            answers = ["NULL"] * 10
        # נוודא שהפורמט מתאים ל-10 שאלות
        answers = (answers + ["NULL"] * 10)[:10]
    else:
        answers = ["NULL"] * 10

    row = [article.get("Title", ""), article.get("Year", ""), abstract] + answers
    ws.append(row)

# שמירת הקובץ
output_path = "sea_level_rise_New_Q1.xlsx"
wb.save(output_path)

print(f"Data has been saved to '{output_path}'")


Total articles found: 60


Error on attempt 1: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Error on attempt 1: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).
Data has been saved to 'sea_level_rise_New_Q1.xlsx'
